## Final Project Submission

Please fill out:
* Student name: Sarabi Eventide 
* Student pace: self-paced
* Scheduled project review date/time: December 18th at 7PM EST 
* Instructor name:  Eli Thomas
* Blog post URL: 


**SCRUB, REREAD, RECONSIDER**
If after scrubbing the previous train of thought still makes sense, go with it. Otherwise.... orz 😭

## Obtaining the Data

Before making any decisions about the data, we must define the scope of this data analysis. In order to do so, I shall consider the following questions:
1. Who are the stakeholders in this problem?
    - The stakeholders are Kings County real estate developers.
2. What do they want to know?
    - They want to know:
        - which neighborhoods yield the highest prices
        - what is considered an "average" house in terms of price, size, age and so on
        - which neighborhoods/houses are likely to increase in price over time (w/ or w/o renovation)
3. What data is available to me?
    - I have a set of data containing information about the size, rooms, location, age, renovation and quality of the houses.
4. How can I answer their questions based on the tools and information available?
    - 
(Steps: obtain, scrub, explore, model, interpret)

After making this decision. I imported the libraries that would be immediately relevant to cleaning and analyzing the data.

In [1]:
import pandas as pd # helps with CSV files
import numpy as np  # helps with number crunching
import matplotlib.pyplot as plt  # helps with visualizing
%matplotlib inline
import seaborn as sns # provides extra functionality with visualizations 

In [2]:
df = pd.read_csv('kc_house_data.csv')  #import the file
df.head()  # taking a look at the first few rows of the file
             # to see what type of information I'm working with 

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


Looking at the excerpt, I see a few different ways we can sort and group the data by date, bedrooms, bathrooms, floors, grade, zipcode and proximity to the waterfront. All of these factors will affect the price of the house. Before I begin re-arranging the table, however, I'd like to look at other descriptive measures to see which groups may be more relevant than others. The new information will also help me clean the data, if necessary. 

## Scrubbing the Data

In [3]:
df.info() # a quick summary of what the rows and columns contain 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
id               21597 non-null int64
date             21597 non-null object
price            21597 non-null float64
bedrooms         21597 non-null int64
bathrooms        21597 non-null float64
sqft_living      21597 non-null int64
sqft_lot         21597 non-null int64
floors           21597 non-null float64
waterfront       19221 non-null float64
view             21534 non-null float64
condition        21597 non-null int64
grade            21597 non-null int64
sqft_above       21597 non-null int64
sqft_basement    21597 non-null object
yr_built         21597 non-null int64
yr_renovated     17755 non-null float64
zipcode          21597 non-null int64
lat              21597 non-null float64
long             21597 non-null float64
sqft_living15    21597 non-null int64
sqft_lot15       21597 non-null int64
dtypes: float64(8), int64(11), object(2)
memory usage: 3.5+ MB


Right off the bat, we see several columns who data types have been miscategorized. Let's change them so that they more accurately reflect the information they represent. First, we need to know whether there are any anomalies that cause the "sqft_basement" column to be labeled as objects and the "yr_renovated" column to be casted as a float.

In [4]:
df.sqft_basement.unique_values()

AttributeError: 'Series' object has no attribute 'unique_values'

In [8]:
df['waterfront'] = df['waterfront'].astype("bool")  # The property either is or isn't facing the waterfront
df['view'] = df['view'].astype("bool")      # The property either has or hasn't been viewed
df['condition'] = df['condition'].astype("str")  # The condition is actually a qualitative variable, not a quantitative one
df['grade'] = df['grade'].astype("str")  # Grade is a qualitative variable
df['yr_built'] = pd.to_datetime(df.yr_built, format = '%Y') # The year is a date, not an integer
df['yr_renovated'] = pd.to_datetime(df.yr_renovated, format = '%Y') 
df['zipcode'] = df['zipcode'].astype("str")   # The zipcode is a categorical variable
df['sqft_basement'] = df['sqft_basement'].astype("int64") # The squarefootage should be quantitative

ValueError: time data 0.0 does not match format '%Y' (match)

The data frame has 21597 rows, most of which contain numerical data. It seems the "waterfront" and "yr_renovated" columns have some missing data, but not enough to render the data unusable. Let's double check the amount of null values. 

In [5]:
df.isna().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

In [6]:
waterfront_percent = round(((2376/21597)*100),2) 
reno_percent = round(((3842/21597)*100),2)
print(f"We are missing {waterfront_percent} percent of the waterfront data and {reno_percent} percent of the household renovation data")

We are missing 11.0 percent of the waterfront data and 17.79 percent of the household renovation data


In other words, we still have about 90% of the waterfront data and about 80% of the household renovation data. These are sufficient numbers to proceed, however, we much also check the data for placeholder values. 

The **.info()** method also shows columns that are encoded as the incorrect data type. Several categorical columns are 
listed as integers when they should be strings. Let's re-encode these columns.

In [6]:
df.describe() # gather summary statistics of the data

,id,price,sqft_living,sqft_lot,sqft_above,lat,long,sqft_living15,sqft_lot15
count,2.159700e+04,2.159700e+04,21597.000000,2.159700e+04,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000
mean,4.580474e+09,5.402966e+05,2080.321850,1.509941e+04,1788.596842,47.560093,-122.213982,1986.620318,12758.283512
std,2.876736e+09,3.673681e+05,918.106125,4.141264e+04,827.759761,0.138552,0.140724,685.230472,27274.441950
min,1.000102e+06,7.800000e+04,370.000000,5.200000e+02,370.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.220000e+05,1430.000000,5.040000e+03,1190.000000,47.471100,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,1910.000000,7.618000e+03,1560.000000,47.571800,-122.231000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,2550.000000,1.068500e+04,2210.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,13540.000000,1.651359e+06,9410.000000,47.777600,-121.315000,6210.000000,871200.000000


**Revisit everything below this line because everything above it has changed**

_______________________________________________________________________________

These summary statistics tell me that we are dealing with a wide range of data, including what appears to be at least one outlier. There appears to be a house contianing 33 bedrooms and 8 bathrooms. I don't know for sure just yet whether these are all in the same house, but it is safe to assume so given that 75% of the houses contain 4 or less bedrooms and 2.5 or less bathrooms. Moreover, the largest house's living room is more than 5 times larger than than houses in the 3rd quartile. We'll need to find and isolate the outliers before looking at other data.

In [9]:
house_bath_count = df.bathrooms.value_counts()  
house_bath_count  # should be a frequency of bathroom counts

2.5     5377
1.0     3851
1.75    3048
2.25    2047
2.0     1930
1.5     1445
2.75    1185
3.0      753
3.5      731
3.25     589
3.75     155
4.0      136
4.5      100
4.25      79
0.75      71
4.75      23
5.0       21
5.25      13
5.5       10
1.25       9
6.0        6
0.5        4
5.75       4
6.75       2
6.25       2
8.0        2
6.5        2
7.75       1
7.5        1
Name: bathrooms, dtype: int64

In [ ]:
house_bath_median = df.bathrooms.median() 
house_bath_median # just checking the median

In [8]:
house_bed_count = df.bedrooms.value_counts()
house_bed_count # should be a frequency of bedroom counts

3     9824
4     6882
2     2760
5     1601
6      272
1      196
7       38
8       13
9        6
10       3
11       1
33       1
Name: bedrooms, dtype: int64

In [10]:
house_bed_median = df.bedrooms.median()
house_bed_median

3.0

The house_bath_ration and house_bed_ratio are value counts that show us how many houses are in each category. 
In this case, the categories are "number of bathrooms" and "number of bedrooms", respectively. The summary statistics above show that the mean number of bathrooms is 2.11 and the mean number of bedrooms is 3.37. These closely mirror the modes of each column, which are 2.50 bathrooms and 3 bedrooms. It seems that the mean number of bathrooms is pulled down by the main set of data, rather than pulled up by the outliers. Because the mean (2.11), median (2.25) and mode (2.50) are all similar, this column is an accurate representation of the data. 